# Last Versian : V3

In [3]:
import datetime

x = datetime.datetime.now()
print(x)

2020-06-17 15:54:04.122149


# Demand Forecast Project

Data sources that will be used are as below:

1- Sales (3 Years history + ongoing) 
2- Product attributes 
3- Holiday

## Demand Forecast generated for 12-16 weeks out will have below dimensions:

Subclass:

Demand Forecast generated for 12-16 weeks out will have below dimensions:

1- Subclass

2- First 3 digits of zip (Digital)

3- Fiscal weeks

4- Loc ID for Walk IN - optional 

5- Holiday - Y/ N - To indicate if the fiscal week had a major US holiday or not

6- Fulfillment types 
    + BOPUS (webex and common)
    + BOSS (webex and common)
    + STH (webex and common)
    + DSV (webex and common)
    + Walk In

7- Product granularity (TBD)



- Frequency of this demand forecast: Weekly

What will it be compared against for gauging? 

Celect

Actuals TY

Method Sales?? - subclass / week (Actuals LY)

![](cover_1600px_web-1280x640.jpg)

# Add Require Libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read data from disk

In [6]:
data = pd.read_csv('Inventory_db.csv' , index_col=('TRAN_DATE' , 'SKU_NBR'))
data.head(5)

/Users/tkma44n/homebrew/Cellar/jupyterlab/2.1.4/libexec/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


DEPT_NBR  SUB_CL_NBR  CL_NBR  ZIPCODE        HOLIDAY  \
TRAN_DATE  SKU_NBR                                                          
2019-12-24 92804569        15          91      90    95122  Christmas Eve   
           92804569        15          91      90    85143  Christmas Eve   
           92804569        15          91      90    94539  Christmas Eve   
           92804569        15          91      90    23231  Christmas Eve   
           92804569        15          91      90    12569  Christmas Eve   

                                 TYPE FULFILLMENT_TYPE  SKU_LNDD_UCST_AMT  \
TRAN_DATE  SKU_NBR                                                          
2019-12-24 92804569  HISTORICAL ECOMM       STH_COMMON               3.19   
           92804569  HISTORICAL ECOMM       STH_COMMON               3.19   
           92804569  HISTORICAL ECOMM       STH_COMMON               3.19   
           92804569  HISTORICAL ECOMM       STH_COMMON               3.19   
           92804569  HISTORICAL ECOMM       STH_COMMON               3.19   

                     GROSS_REVENUE  FSCL_WK_NBR  GROSS_QTY_SOLD  
TRAN_DATE  SKU_NBR                                               
2019-12-24 92804569          11.72           47               4  
           92804569           5.58           47               2  
           92804569           4.88           47               2  
           92804569           3.99           47               1  
           92804569           8.78           47               3

In [7]:
data['DEPT_CL_SUB']=data['DEPT_NBR'].astype(str) + data['CL_NBR'].astype(str) +data['SUB_CL_NBR'].astype(str)
data['DEPT_CL_SUB'] = data['DEPT_CL_SUB'].astype(int)

In [8]:
data.DEPT_CL_SUB.value_counts().head(20)

2656061    38518
159091     32809
2654041    32652
1222026    26317
2292022    22393
7154041    22045
664040     20601
358081     20529
664043     20102
744045     19871
9833036    18031
1532021    17835
556062     17757
558081     17562
2308081    16988
3274041    15775
3522021    15107
159092     14910
3115056    13759
159093     13141
Name: DEPT_CL_SUB, dtype: int64

# Seperate each digit of zip code


In [6]:

# make string version of original column, call it 'col'
data['ZIPCODE'] = data['ZIPCODE'].astype(str)

# make the new columns using string indexing
# data['ZIPCODE_1'] = data['ZIPCODE'].str[0:1].astype(int)
# data['ZIPCODE_2'] = data['ZIPCODE'].str[1:2].astype(int)
# data['ZIPCODE_3'] = data['ZIPCODE'].str[2:3].astype(int)
data['three_DIGZIP'] = data['ZIPCODE'].str[0:3].astype(int)
# # get rid of the extra variable (if you want)
# df.drop('col', axis=1, inplace=True)


In [7]:
del data['ZIPCODE'] , data['DEPT_NBR'] , data['SUB_CL_NBR'] , data['CL_NBR']

In [8]:
data.three_DIGZIP.value_counts()

604    21909
117    20260
750    18731
605    18388
601    17600
       ...  
418        2
994        2
692        2
429        1
568        1
Name: three_DIGZIP, Length: 844, dtype: int64

In [9]:
data.FSCL_WK_NBR.unique()

array([47])

# Present the Unique value of each colomn


In [10]:
def find_unique_value(col):
    res=data[col].value_counts().sort_index()
    print(res)
    res.plot.bar()

In [11]:
find_unique_value("FULFILLMENT_TYPE")

DSV_COMMON       7685
DSV_WEBEX       40051
STH_COMMON      76654
STH_WEBEX       23392
WALK-IN       1711173
Name: FULFILLMENT_TYPE, dtype: int64


In [12]:
data.describe()

SKU_LNDD_UCST_AMT  GROSS_REVENUE  FSCL_WK_NBR  GROSS_QTY_SOLD  \
count       1.858952e+06   1.858955e+06    1858955.0    1.858955e+06   
mean        1.377387e+01   3.356988e+01         47.0    1.493408e+00   
std         2.360390e+01   3.752351e+02          0.0    3.748606e+00   
min         0.000000e+00   0.000000e+00         47.0    0.000000e+00   
25%         4.000000e+00   6.350000e+00         47.0    1.000000e+00   
50%         8.460000e+00   1.517000e+01         47.0    1.000000e+00   
75%         1.710000e+01   3.199000e+01         47.0    1.000000e+00   
max         5.950000e+03   4.191250e+05         47.0    2.500000e+03   

        DEPT_CL_SUB  three_DIGZIP  
count  1.858955e+06  1.858955e+06  
mean   2.879193e+06  5.374705e+02  
std    2.823205e+06  2.586656e+02  
min    1.120240e+05  1.000000e+02  
25%    6.580830e+05  3.100000e+02  
50%    2.244047e+06  5.250000e+02  
75%    4.193034e+06  7.730000e+02  
max    9.833037e+06  9.990000e+02

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1858955 entries, ('2019-12-24', 92804569) to ('2019-12-28', 98750609)
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   HOLIDAY            object 
 1   TYPE               object 
 2   FULFILLMENT_TYPE   object 
 3   SKU_LNDD_UCST_AMT  float64
 4   GROSS_REVENUE      float64
 5   FSCL_WK_NBR        int64  
 6   GROSS_QTY_SOLD     int64  
 7   DEPT_CL_SUB        int64  
 8   three_DIGZIP       int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 137.0+ MB


# Find missing data
## check null value in all variables

In [14]:
data.isnull().sum()

HOLIDAY              1422764
TYPE                       0
FULFILLMENT_TYPE           0
SKU_LNDD_UCST_AMT          3
GROSS_REVENUE              0
FSCL_WK_NBR                0
GROSS_QTY_SOLD             0
DEPT_CL_SUB                0
three_DIGZIP               0
dtype: int64

# Apply LabelEncoder

## 1. HOLIDAY

In [15]:
data['HOLIDAY'] = data['HOLIDAY'].replace(np.nan , 'Not Holiday' , regex=True)
data['HOLIDAY'].unique()

array(['Christmas Eve', 'Not Holiday', 'Christmas Day'], dtype=object)

In [16]:
Holiday = ['Christmas Eve', 'Christmas Day']

data['HOLIDAY']= data['HOLIDAY'].replace(to_replace =Holiday, 
                 value ="Holiday") 
data['HOLIDAY'].value_counts()

Not Holiday    1422764
Holiday         436191
Name: HOLIDAY, dtype: int64

In [35]:
data_Holiday_FSCL_WK_NBR=data.loc[data['HOLIDAY'] == 'Holiday', 'FSCL_WK_NBR']
data_Holiday_FSCL_WK_NBR.unique().tolist()

[47]

In [ ]:
# data['HOLIDAY'] = data['HOLIDAY'].replace('47' , 'Holiday_FC_WE' , regex=True)
# data['HOLIDAY'].unique()

In [17]:
from sklearn import preprocessing 
label_encod = preprocessing.LabelEncoder()
data['HOLIDAY'] = label_encod.fit_transform(data['HOLIDAY'])
data['HOLIDAY'].unique()
# Holiday = 0
# not Holiday = 1

array([0, 1])

## 2. Type

In [18]:
data['TYPE'].value_counts()

HISTORICAL B&M      1711173
HISTORICAL ECOMM     147782
Name: TYPE, dtype: int64

In [19]:
from sklearn import preprocessing 
label_encod = preprocessing.LabelEncoder()
data['TYPE'] = label_encod.fit_transform(data['TYPE'])
data['TYPE'].unique()
# HISTORICAL ECOMM = 0
# HISTORICAL B&M = 1

array([1, 0])

## FULFILLMENT_TYPE


In [20]:
# replace 'DSV_WEBEX', 'STH_COMMON', 'BOPUS_COMMON', 'BOPUS', 'STH_WEBEX', 
# 'DSV_COMMON', 'BOSS_COMMON', 'BOPUS_WEBEX',  'BOSS_WEBEX'  to webex and common

Digital = ['DSV_WEBEX', 'STH_COMMON', 'BOPUS_COMMON', 'BOPUS',
       'STH_WEBEX', 'DSV_COMMON', 'BOSS_COMMON', 'BOPUS_WEBEX',
       'BOSS_WEBEX']

data['FULFILLMENT_TYPE']= data['FULFILLMENT_TYPE'].replace(to_replace =Digital, 
                 value ="Digital") 
data['FULFILLMENT_TYPE'].value_counts()

WALK-IN             1711173
webex_and_common     147782
Name: FULFILLMENT_TYPE, dtype: int64

In [42]:
label_encod = preprocessing.LabelEncoder()
data['FULFILLMENT_TYPE'] = label_encod.fit_transform(data['FULFILLMENT_TYPE'])
data['FULFILLMENT_TYPE'].value_counts()

# WALK-IN = 0
# webex_and_common = 1

0    1711173
1     147782
Name: FULFILLMENT_TYPE, dtype: int64

# Drop SKU_LNDD_UCST_AMT with null

In [156]:
data = data[data['SKU_LNDD_UCST_AMT'].notna()]

In [157]:
data.isnull().sum()

DEPT_NBR             0
SUB_CL_NBR           0
CL_NBR               0
HOLIDAY              0
TYPE                 0
FULFILLMENT_TYPE     0
SKU_LNDD_UCST_AMT    0
GROSS_REVENUE        0
FSCL_WK_NBR          0
GROSS_QTY_SOLD       0
ZIPCODE_1            0
ZIPCODE_2            0
ZIPCODE_3            0
dtype: int64

# Display our Traget point

In [158]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.countplot(x="GROSS_QTY_SOLD" , data=data , palette='BrBG')

In [159]:
print(list(data))

['DEPT_NBR', 'SUB_CL_NBR', 'CL_NBR', 'HOLIDAY', 'TYPE', 'FULFILLMENT_TYPE', 'SKU_LNDD_UCST_AMT', 'GROSS_REVENUE', 'FSCL_WK_NBR', 'GROSS_QTY_SOLD', 'ZIPCODE_1', 'ZIPCODE_2', 'ZIPCODE_3']


# Reordring column

In [160]:
col=['DEPT_NBR', 'SUB_CL_NBR', 'CL_NBR', 'HOLIDAY', 'TYPE', 'FULFILLMENT_TYPE', 
     'SKU_LNDD_UCST_AMT', 'GROSS_REVENUE',
     'FSCL_WK_NBR', 'ZIPCODE_1', 'ZIPCODE_2', 'ZIPCODE_3' , 'GROSS_QTY_SOLD'] # ->> Target
data = data.reindex(columns=col)
data.head()

DEPT_NBR  SUB_CL_NBR  CL_NBR  HOLIDAY  TYPE  \
TRAN_DATE  SKU_NBR                                                 
2019-12-23 99436430       155          56      50        1     0   
           94439586       827          43      40        1     0   
2019-12-28 98326608       715          41      40        1     0   
           91749019       266          74      70        1     0   
2019-12-24 90005584       983          35      30        0     0   

                     FULFILLMENT_TYPE  SKU_LNDD_UCST_AMT  GROSS_REVENUE  \
TRAN_DATE  SKU_NBR                                                        
2019-12-23 99436430                 0              12.08          16.78   
           94439586                 0              15.00          44.99   
2019-12-28 98326608                 0               2.50          24.23   
           91749019                 0               4.50           0.00   
2019-12-24 90005584                 0               0.00          75.00   

                     FSCL_WK_NBR  ZIPCODE_1  ZIPCODE_2  ZIPCODE_3  \
TRAN_DATE  SKU_NBR                                                  
2019-12-23 99436430           47          6          1          1   
           94439586           47          1          4          5   
2019-12-28 98326608           47          3          6          3   
           91749019           47          2          3          3   
2019-12-24 90005584           47          1          7          6   

                     GROSS_QTY_SOLD  
TRAN_DATE  SKU_NBR                   
2019-12-23 99436430               1  
           94439586               1  
2019-12-28 98326608               5  
           91749019               1  
2019-12-24 90005584               1

# Normalize our DATA

In [161]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
data['SKU_LNDD_UCST_AMT'] = scaler.fit_transform(data[['SKU_LNDD_UCST_AMT']])
scaler = MinMaxScaler() 
data['GROSS_REVENUE'] = scaler.fit_transform(data[['GROSS_REVENUE']])

In [162]:
data.describe()

DEPT_NBR    SUB_CL_NBR        CL_NBR       HOLIDAY          TYPE  \
count  1.858952e+06  1.858952e+06  1.858952e+06  1.858952e+06  1.858952e+06   
mean   2.874236e+02  5.225557e+01  4.908107e+01  7.653565e-01  7.949748e-02   
std    2.823598e+02  2.339755e+01  2.338014e+01  4.237759e-01  2.705137e-01   
min    1.100000e+01  1.000000e+01  1.000000e+01  0.000000e+00  0.000000e+00   
25%    6.500000e+01  3.200000e+01  3.000000e+01  1.000000e+00  0.000000e+00   
50%    2.240000e+02  4.800000e+01  4.000000e+01  1.000000e+00  0.000000e+00   
75%    4.190000e+02  7.100000e+01  7.000000e+01  1.000000e+00  0.000000e+00   
max    9.830000e+02  9.900000e+01  9.000000e+01  1.000000e+00  1.000000e+00   

       FULFILLMENT_TYPE  SKU_LNDD_UCST_AMT  GROSS_REVENUE  FSCL_WK_NBR  \
count      1.858952e+06       1.858952e+06   1.858952e+06    1858952.0   
mean       7.949748e-02       2.314935e-03   8.009528e-05         47.0   
std        2.705137e-01       3.967042e-03   8.952829e-04          0.0   
min        0.000000e+00       0.000000e+00   0.000000e+00         47.0   
25%        0.000000e+00       6.722689e-04   1.515061e-05         47.0   
50%        0.000000e+00       1.421849e-03   3.619445e-05         47.0   
75%        0.000000e+00       2.873950e-03   7.632568e-05         47.0   
max        1.000000e+00       1.000000e+00   1.000000e+00         47.0   

          ZIPCODE_1     ZIPCODE_2     ZIPCODE_3  GROSS_QTY_SOLD  
count  1.858952e+06  1.858952e+06  1.858952e+06    1.858952e+06  
mean   4.920678e+00  4.189150e+00  3.511344e+00    1.493409e+00  
std    2.598353e+00  2.906855e+00  2.838863e+00    3.748609e+00  
min    1.000000e+00  0.000000e+00  0.000000e+00    0.000000e+00  
25%    3.000000e+00  2.000000e+00  1.000000e+00    1.000000e+00  
50%    5.000000e+00  4.000000e+00  3.000000e+00    1.000000e+00  
75%    7.000000e+00  7.000000e+00  6.000000e+00    1.000000e+00  
max    9.000000e+00  9.000000e+00  9.000000e+00    2.500000e+03

In [163]:
data.DEPT_NBR.unique()

array([155, 827, 715, 266, 983,  15, 552,  55, 265,  73, 152, 139, 151,
        52,  74, 833, 533, 627, 122, 920,  12, 446,  76,  63, 135, 927,
       235,  89, 115,  35, 722, 229,  36,  66, 333, 422, 231, 327, 228,
       224, 443, 230, 156, 133,  39, 239, 311,  27, 352,  25, 456,  26,
       157,  59, 834, 153, 963, 852, 511, 232, 614, 420, 527, 419, 259,
       129,  60,  17,  37, 187, 566,  43, 226, 853, 915, 922, 233,  46,
        31, 524,  71,  67,  32, 111, 223, 624, 464,  29,  49,  56, 611,
       287, 218, 727, 315, 411, 256,  24, 251, 622, 211, 964, 657, 318,
       245, 960,  34, 514, 316, 364, 165, 412, 452, 146, 127,  65, 164,
       556, 413, 441, 911, 355, 371, 121, 949,  23, 356,  11, 116,  47,
       906, 939, 246, 917, 932, 210, 543, 928, 966, 976, 405, 134,  97,
       902, 743, 644, 113,  48, 656, 427, 944, 197,  33, 440,  13, 363,
       903, 918,  41, 314,  20, 961, 571, 252, 213, 353, 227, 263,  61,
       923, 457, 713, 975,  95, 117, 424, 219,  75, 357, 244, 94

In [164]:
data.to_csv("data_prepared.csv",index=False)

In [ ]:
"Done"